# Thai Wiki Language Model for Text Generation

This notebook details how you can use pretrained language model on [Thai Wikipedia Dump](https://dumps.wikimedia.org/thwiki/latest/thwiki-latest-pages-articles.xml.bz2) to generate texts.

In [1]:
# #uncomment if you are running from google colab
# !pip install sklearn_crfsuite
# !pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
# !pip install fastai==1.0.45

In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm import tqdm_notebook
from collections import Counter
import re

#viz
import matplotlib.pyplot as plt
import seaborn as sns

#fastai
import fastai
from fastai.text import *
from fastai.callbacks import CSVLogger

#pythainlp
from pythainlp.ulmfit import *

Download : wiki_lm_lstm
re-download
from wiki_lm_lstm 0.31 update to wiki_lm_lstm 0.31
yes or no (y / n) : y


thwiki_lm.pth?dl=1: 1.05GB [01:28, 11.8MB/s]                                


Download : wiki_itos_lstm
re-download
from wiki_itos_lstm 0.31 update to wiki_itos_lstm 0.31
yes or no (y / n) : y


thwiki_itos.pkl?dl=1: 1.53MB [00:02, 743kB/s]                             


In [3]:
#get dummy data
imdb = untar_data(URLs.IMDB_SAMPLE)
dummy_df = pd.read_csv(imdb/'texts.csv')

#get vocab
thwiki_itos = pickle.load(open(_THWIKI_LSTM['itos_fname'],'rb'))
thwiki_vocab = fastai.text.transform.Vocab(thwiki_itos)

#dummy databunch
tt = Tokenizer(tok_func = ThaiTokenizer, lang = 'th', pre_rules = pre_rules_th, post_rules=post_rules_th)
processor = [TokenizeProcessor(tokenizer=tt, chunksize=10000, mark_fields=False),
            NumericalizeProcessor(vocab=thwiki_vocab, max_vocab=60000, min_freq=3)]
data_lm = (TextList.from_df(dummy_df, imdb, cols=['text'], processor=processor)
    .random_split_by_pct(0.2)
    .label_for_lm()
    .databunch(bs=64))


data_lm.sanity_check()

In [4]:
#check vocab size
len(data_lm.vocab.itos)

60004

In [5]:
config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False, tie_weights=True, out_bias=True,
             output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)
trn_args = dict(drop_mult=0.9, clip=0.12, alpha=2, beta=1)

learn = language_model_learner(data_lm, AWD_LSTM, config=config, pretrained=False, **trn_args)

#load pretrained models
learn.load_pretrained(**_THWIKI_LSTM)

In [6]:
learn.predict('กาลครั้งหนึ่งนานมาแล้ว ', 200, temperature=0.8, sep = '')

'กาลครั้งหนึ่งนานมาแล้ว เกิดอาการปวดศีรษะ ความขัดแย้งในครอบครัว ความถึงแก่อสัญกรรมของบิดามารดา ได้สร้างความตื่นตระหนกให้แก่ลูกหลานมาก โดยเด็กทั้งสองลงท้ายด้วยคำว่า "แม่" เข้าด้วยกัน และและคำว่า "ลูก" เป็นคำที่มาจากคำว่า "ลูก" ในภาษาสันสกฤต หมายถึงแม่ ซึ่งหมายถึงครอบครัว และ "ความอุดมสมบูรณ์" \n \n \n \n =2444 (วงดนตรี)= \n 2537 (วงดนตรี) \n \n สถาบันดนตรี (Music Academy) เป็นคณะแรกในจุฬาลงกรณ์มหาวิทยาลัย ก่อตั้งเมื่อปี พ.ศ. 2514 ในชื่อ "คณะนิเทศศาสตร์และการบัญชี" โดยมีผลงานจากริมดวงจันทร์ที่ประสบความสำเร็จอย่างมากทั้งในและนอกประเทศ การเผยแพร่เพลงและวัฒนธรรมของคนไทย และในสื่อต่างๆ ที่เปิดให้ทุกคนในประเทศไทยเข้าใจในดนตรีแนวนี้ เป็นที่รู้จักของผู้ชมทั่วไป และความนิยมในแนวดนตรีที่ไม่เหมือนใคร \n \n ประวัติ'